In [1]:
import os
os.path.exists

<function genericpath.exists(path)>

In [2]:
import os
if not os.path.exists('enwiki-latest-pages-articles.xml.bz2'):
    download_str = '''#!/bin/sh
set -e

LG=$1
WIKI_DUMP_NAME=${LG}wiki-latest-pages-articles.xml.bz2
WIKI_DUMP_DOWNLOAD_URL=https://dumps.wikimedia.org/${LG}wiki/latest/$WIKI_DUMP_NAME

# download latest Wikipedia dump in chosen language
echo "Downloading the latest $LG-language Wikipedia dump from $WIKI_DUMP_DOWNLOAD_URL..."
wget -c $WIKI_DUMP_DOWNLOAD_URL
echo "Succesfully downloaded the latest $LG-language Wikipedia dump to $WIKI_DUMP_NAME"
    '''
    with open('download_wiki_dump.sh', 'w') as f:
        f.write(download_str)
    !chmod +777 download_wiki_dump.sh
    !./download_wiki_dump.sh en

In [ ]:
import os
if not os.path.exists('enwiki-latest-pages-articles.xml.bz2'):
    processing_str = '''
#!/bin/sh
set -e

WIKI_DUMP_FILE_IN=$1
WIKI_DUMP_FILE_OUT=${WIKI_DUMP_FILE_IN%%.*}.txt

# clone the WikiExtractor repository
git clone https://github.com/attardi/wikiextractor.git

# extract and clean the chosen Wikipedia dump
echo "Extracting and cleaning $WIKI_DUMP_FILE_IN to $WIKI_DUMP_FILE_OUT..."
python3 wikiextractor/WikiExtractor.py $WIKI_DUMP_FILE_IN --processes 8 -q -o - \
| sed "/^\s*\$/d" \
| grep -v "^<doc id=" \
| grep -v "</doc>\$" \
> $WIKI_DUMP_FILE_OUT
    echo "Succesfully extracted and cleaned $WIKI_DUMP_FILE_IN to $WIKI_DUMP_FILE_OUT"
'''
    with open('extract_and_clean_wiki_dump.sh', 'w') as f:
        f.write(processing_str)
    !chmod +777 extract_and_clean_wiki_dump.sh
    !./extract_and_clean_wiki_dump.sh enwiki-latest-pages-articles.xml.bz2

In [3]:
from pathlib import Path


In [6]:
Path('download_wiki_dump.sh').parent

PosixPath('.')

In [ ]:
#blingfire text_to_sentence

#!/usr/bin/env python
# -*- coding: utf-8 -*-
import sys
from pathlib import Path
from blingfire import text_to_sentences

def use_blingfire(text_input):
    wiki_dump_file_in = Path(text_input)
    wiki_dump_file_out = wiki_dump_file_in.parent / \
        f'{wiki_dump_file_in.stem}_preprocessed{wiki_dump_file_in.suffix}'

    print(f'Pre-processing {wiki_dump_file_in} to {wiki_dump_file_out}...')
    with open(wiki_dump_file_out, 'w', encoding='utf-8') as out_f:
        with open(wiki_dump_file_in, 'r', encoding='utf-8') as in_f:
            for line in in_f:
                sentences = text_to_sentences(line)
                out_f.write(sentences + '\n')
    print(f'Successfully pre-processed {wiki_dump_file_in} to {wiki_dump_file_out}...')


if 1:
    use_blingfire('enwiki-latest-pages-articles.txt')
